In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
train_id = train['id']
test_id = test['id']

In [ ]:
train.drop(columns = ['id'], inplace = True)
test.drop(columns = ['id'], inplace = True)

Location column has a lot of missing values and keyword column contains information that is present in the text column and we will be exracting that information down the line from the text column

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.drop(columns = ['keyword','location'], inplace = True)
test.drop(columns = ['keyword','location'], inplace = True)

In [ ]:
train.head()

## Data Cleaning

In [ ]:
# Converting all text to lowercase
train['text'] = [t.lower() for t in train['text']]
test['text'] = [t.lower() for t in test['text']]

In [ ]:
# Removing punctuations
import re
import string
train['text'] = [re.sub('[%s]' % re.escape(string.punctuation), '', i) for i in train['text']]
test['text'] = [re.sub('[%s]' % re.escape(string.punctuation), '', i) for i in test['text']]

In [ ]:
# Removing numeric characters
train['text'] = [re.sub('\d','',n) for n in train['text']]
test['text'] = [re.sub('\d','',n) for n in test['text']]

## Preprocessing Text Data

### Tokenization and Stop Words

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
# Word Tokenization

train['text'] = [word_tokenize(i) for i in train['text']]
test['text'] = [word_tokenize(i) for i in test['text']]

In [ ]:
train['text'].head()

In [ ]:
# Stop Words Removal

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
train['text'] = [[i for i in j if not i in stop_words] for j in train['text']]
test['text'] = [[i for i in j if not i in stop_words] for j in test['text']]

In [ ]:
train.head()

### Lemmatization

In [ ]:
from collections import defaultdict
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

tag_map

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

train['text'] = [[lemmatizer.lemmatize(word, tag_map[tag[0]]) for word, tag in pos_tag(i)] for i in train['text']]
test['text'] = [[lemmatizer.lemmatize(word, tag_map[tag[0]]) for word, tag in pos_tag(i)] for i in test['text']]

In [ ]:
train.head()

In [ ]:
train['lemmatized_text'] = train['text'].apply(lambda x : ' '.join(x))
test['lemmatized_text'] = test['text'].apply(lambda x : ' '.join(x))

In [ ]:
train.head()

In [ ]:
train.drop(columns = ['text'], inplace = True)
test.drop(columns = ['text'], inplace = True)

### Word Embedding using TF_IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 5000)

train_emb = tfidf.fit_transform(train['lemmatized_text']).toarray()
test_emb = tfidf.fit_transform(test['lemmatized_text']).toarray()

In [ ]:
train_emb.shape[1:]

In [ ]:
y = train['target']

## Model Training

### Naive Bayes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [ ]:
MNB = MultinomialNB()

Splitting the train set in train and validation set to see how good is Naive Bayes for our data

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(train_emb,y,test_size = 0.3, random_state = 100) 

In [ ]:
MNB.fit(x_train,y_train)
pred_MNB = MNB.predict(x_valid)

In [ ]:
print("Accuracy score : {:.2f}".format(accuracy_score(y_valid, pred_MNB)))

In [ ]:
print("ROC-AUC score : {:.2f}".format(roc_auc_score(y_valid, pred_MNB)))

In [ ]:
print(classification_report(y_valid, pred_MNB))

### Model Training

In [ ]:
MNB.fit(train_emb,y)

In [ ]:
MNB_predictions = MNB.predict(test_emb)

In [ ]:
Prediction_results = pd.DataFrame({"target": MNB_predictions}, index = test_id)

In [ ]:
#submission_file = Prediction_results.to_csv('submission.csv')

#### Multinomial Naive Bayes got a 0.515 score which is decent but can be significantly improved.

### Support Vector Machines

In [ ]:
from sklearn import svm
SVC = svm.SVC()
#SVC.fit(x_train,y_train)
#pred_SVC = SVC.predict(x_valid)

In [ ]:
#print("Accuracy score : {:.2f}".format(accuracy_score(y_valid, pred_SVC)))

In [ ]:
#print("ROC-AUC score : {:.2f}".format(roc_auc_score(y_valid, pred_SVC)))

## Alternative Approach using Sequencing, Padding, and LSTM

### Sequencing and Sentence Padding

In [ ]:
from collections import Counter

# Finding the number of unique word in the corpus
def word_counter(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [ ]:
train.head()

In [ ]:
train_text = train.lemmatized_text
counter = word_counter(train_text)
counter

In [ ]:
print("Number of unique words in the corpus : {:.2f}".format(len(counter)))

In [ ]:
words = len(counter)
# maximum number of words in a sequence
max_length = 20

In [ ]:
train_sent = train['lemmatized_text']
train_labels = train['target']
test_sent = test['lemmatized_text']

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=words)
tokenizer.fit_on_texts(train_sent)

In [ ]:
word_index = tokenizer.word_index
word_index

In [ ]:
train_sequence = tokenizer.texts_to_sequences(train_sent)

In [ ]:
train_sequence[0]

In [ ]:
test_sequence = tokenizer.texts_to_sequences(test_sent)

In [ ]:
test_sequence

#### Sequence Padding

In [ ]:
from keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(train_sequence, maxlen = max_length, padding = "post", truncating = "post")

In [ ]:
train_padded

In [ ]:
test_padded = pad_sequences(test_sequence, maxlen = max_length, padding = "post", truncating = "post")

In [ ]:
test_padded

### Model Building using LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam
import tensorflow as tf

def leaky_relu(z, name = None):
    return tf.maximum(0.01*z,z, name = name)

model = Sequential()

model.add(Embedding(words,32,input_length = max_length))
#model.add(LSTM(128, return_sequences = True, dropout = 0.1))
model.add(LSTM(64, dropout = 0.1))
model.add(Dense(units = 32 , activation = leaky_relu))
model.add(Dense(1, activation = tf.nn.elu))

optimizer = Adam(learning_rate = 3e-4)

model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_padded, train_labels, epochs = 40)

In [ ]:
import h5py
#model.save('baseline_lstm_model.h5')

In [ ]:
from keras.models import load_model
#model = load_model('baseline_lstm_model.h5')

In [ ]:
lstm_base_pred = model.predict_classes(test_padded, verbose = 0)

In [ ]:
lstm_base_pred = lstm_base_pred.reshape(-1,1).ravel()

In [ ]:
len(lstm_base_pred)

In [ ]:
Prediction_results_lstm = pd.DataFrame({"target":lstm_base_pred}, index = test_id)
Prediction_results_lstm

In [ ]:
#submission_lstm_elu_leaky_relu = Prediction_results_lstm.to_csv('submission_lstm_elu_leaky_relu.csv')